In [1]:
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import numpy as np
import bqplot as bq
import time
import threading
from scipy.integrate import odeint

In [2]:
flag = True
tiempo_span = 10

k=0.1
#condición inicial
h0 = 3                 # Altura inicial h0
Qi0 = k*np.sqrt(h0) #cálculo del caudal de equilibrio
print('Caudal de equilibrio Qi0= ',Qi0)


tiempo = np.linspace(-tiempo_span,0,500)
dt = abs(tiempo[1]-tiempo[0])
y = np.ones(len(tiempo))*h0
Qi = np.ones(len(tiempo))*Qi0#input model

Caudal de equilibrio Qi0=  0.17320508075688773


In [3]:
b_stop = Button(
    description = 'Stop',
    icon = 'stop',
    button_style = 'warning',
    layout =  Layout(width='100px')
)

def stop_click(b):
    global flag
    flag = False
    
b_stop.on_click(stop_click)

w1 = FloatProgress(
    value = y[-1],
    min=0,
    max=10,
    description = 'PV:',
    style ={'description_width':'initial'},
    layout=Layout(width='40px'),
    orientation='vertical'
)

w2 = Label(
    value=str(np.round(y[-1],2)),
    layout=Layout(width='50px',margin='0 10px 30px')
)

w12 = HBox(
    children=(w1,w2),
    layout = Layout(margin='0 0 0 10px')
)
#Barra de caudal y texto
QiProg = FloatProgress(
    value = Qi[-1],
    min=0,
    max=1,
    description = 'Qi:',
    style ={'description_width':'initial'},
    layout=Layout(width='40px'),
    orientation='vertical'
)

QiLabel = Label(
    value=str(np.round(Qi[-1],3)),
    layout=Layout(width='50px',margin='0 10px 30px')
)

QiPL = HBox(
    children=(QiProg,QiLabel),
    layout = Layout(margin='0 0 0 10px')
)

wA = FloatSlider(
    value=Qi0,
    min= 0,
    max= 1,
    step =0.005,
    description='Valve Input:',
    layout=Layout(width='490px',margin='0 0 5px 0')
)

In [4]:
#firgura de Nivel(t)

x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

x_ax = bq.Axis(
    label='time [s]',
    scale=x_sc
)

y_ax = bq.Axis(
    label='Level [m]',
    scale=y_sc,
    orientation= 'vertical'
)

Line = bq.Lines(
    x=tiempo,
    y=y,
    scales={'x':x_sc,'y':y_sc}
)

fig = bq.Figure(
    layout=Layout(width='500px',height='300px'),
    axes=[x_ax,y_ax],
    marks= [Line],
    fig_margin = dict(top=10,bottom=40,left=50, right=10)
)

In [5]:
#firgura de Nivel(t)

x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

x_ax = bq.Axis(
    label='time [s]',
    scale=x_sc
)

y_ax = bq.Axis(
    label='Caudal [m³/s]',
    scale=y_sc,
    orientation= 'vertical'
)

Line2 = bq.Lines(
    x=tiempo,
    y=Qi,
    line_style='dashed',
    scales={'x':x_sc,'y':y_sc}
)

fig_Q = bq.Figure(
    layout=Layout(width='500px',height='300px'),
    axes=[x_ax,y_ax],
    marks= [Line2],
    fig_margin = dict(top=10,bottom=40,left=50, right=10)
)

In [6]:
box1 = HBox(
    children=(fig,w12),
    layout=Layout(border='solid 2px gray',width='530px')
)
box2 = HBox(
    children=(fig_Q,QiPL),
    layout=Layout(border='solid 2px gray',width='530px')
)

app =VBox(
    children=(b_stop,wA,box1,box2)
)

In [7]:
# define energy balance model
def tank(x, t, u):
    # Parameters
    k= 0.1
    A=1
    h_min = 0  # h mínimo no puede ser negativo, por defecto h debe ser mayor a 0
    h_max = 10 # h máximo debe ser mayor a h_max...por defecto a 10m 

    # variables de estado
    h = x[0]

    # Cálculo de la acción de control
    Qi = u if u > 0 else 0 # solo caudales positivos

    #calculo de la derivada de h... dh/dt
    h_sat = max(h_min, min(h, h_max))
    dh = (Qi - k*np.sqrt(h_sat))/A
    return dh

In [8]:
def work():
    global tiempo #tiempo de simulación
    global y
    global Qi
    
    prev_time = time.time()
    sleep_max = .5
    while flag:
        # Record time and change in time
        t = time.time()
        dt = t - prev_time
        prev_time = t
        tiempo = np.delete(tiempo, 0)
        tiempo = np.append(tiempo,tiempo[-1]+dt)

        #calculo input
        Qi = np.delete(Qi, 0)
        A = wA.value
        Qi = np.append(Qi,A)
        
        #Calculo de nivel para u=Qi
        y = np.delete(y, 0)
        level = odeint(tank,y[-1],[0,dt],args=(Qi[-1],))
        y = np.append(y,level[-1])
         

        #actualización de widgets
        w1.value = y[-1]
        w2.value = str(np.round(y[-1],3))
        
        Line.x = tiempo
        Line.y = y

        QiProg.value = Qi[-1]
        QiLabel.value = str(np.round(Qi[-1],3))
        
        Line2.x = tiempo
        Line2.y = Qi

        # Sleep time
        sleep = sleep_max - (time.time() - prev_time)
        if sleep>=0.01:
            time.sleep(sleep-0.01)
        else:
            time.sleep(0.01)        

In [9]:
flag = True

thread = threading.Thread(target=work)

display(app)

thread.start()


In [10]:
import matplotlib.pyplot as plt


In [11]:
np.shape(y)

(500,)

In [12]:
level


NameError: name 'level' is not defined

In [ ]:
level = odeint(tank,3,[0,.5],args=(.18,))

In [ ]:
level
